In [0]:
from mxnet import gpu
from mxnet import cpu
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
from mxnet import init
from time import time

import utils


def train(num_gpus, batch_size, lr):
    train_data, test_data = utils.load_data_fashion_mnist(batch_size)

    ctx = [gpu(i) for i in range(num_gpus)]
    print('Running on', ctx)

    net = utils.resnet18_28(10)
    net.initialize(init=init.Xavier(), ctx=ctx)
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    trainer = gluon.Trainer(net.collect_params(),'sgd', {'learning_rate': lr})

    for epoch in range(5):
        start = time()
        total_loss = 0
        for data, label in train_data:
            data_list = gluon.utils.split_and_load(data, ctx)
            label_list = gluon.utils.split_and_load(label, ctx)
            with autograd.record():
                losses = [loss(net(x), y) for x, y in zip(data_list, label_list)]
            for l in losses:
                l.backward()
            total_loss += sum([l.sum().asscalar() for l in losses])
            trainer.step(batch_size)

        nd.waitall()
        print('Epoch %d, training time = %.1f sec'%(
            epoch, time()-start))

        test_acc = utils.evaluate_accuracy(test_data, net, ctx[0])
        print('         validation accuracy = %.4f'%(test_acc))

In [12]:
train(1, 256, .1)

Running on [gpu(0)]
Epoch 0, training time = 42.6 sec
         validation accuracy = 0.8919
Epoch 1, training time = 42.3 sec
         validation accuracy = 0.9018
Epoch 2, training time = 42.8 sec
         validation accuracy = 0.9008
Epoch 3, training time = 43.4 sec
         validation accuracy = 0.9112
Epoch 4, training time = 43.3 sec
         validation accuracy = 0.9086
